<a href="https://colab.research.google.com/github/Varfalamei/cross-domain-recsys/blob/lab3/notebook/collect_data_fit_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!gdown --id 1NM4uSjSg0e2LPBT_aC0v1XaRRh932jxJ
!gdown --id 19IDNSFnAh-JiVx05BSJJRmU-eTdzlG1T
!gdown --id 1j9DrLV4spxTqeQ_OVPDOwsCGHjcZcKjU
!gdown --id 1-08GhVIT-LcsDodcVaVep7LJeaCCbGfx # description_emb_books.csv
!gdown --id 1-MZoIhuMe6j5q48HDCe7AG9MTL5-mKkM # description_emb_films.csv
!gdown --id 1-DmeIZyRYoynique2JPntt17SZCAk_az # TF IDF features

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1NM4uSjSg0e2LPBT_aC0v1XaRRh932jxJ
To: /content/books.csv
100% 69.0M/69.0M [00:00<00:00, 105MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=19IDNSFnAh-JiVx05BSJJRmU-eTdzlG1T
To: /content/films.csv
100% 28.6M/28.6M [00:00<00:00, 82.4MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://

In [2]:
!pip install catboost
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 7.8 MB/s 
     |████████████████████████████████| 81 kB 11.3 MB/s 
     |████████████████████████████████| 209 kB 60.9 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 147 kB 67.1 MB/s 
     |████████████████████████████████| 112 kB 55.2 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=11dd4692c8b6d0f6c5df16aed61f098440821c0a937b2783ef760758dbd1fbd5
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [4]:
import numpy as np
import pandas as pd

import optuna

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

import warnings
warnings.filterwarnings('always')

### Сбор датасета

#### Подготовка таргета

In [5]:
film_adaptations = pd.read_csv('film_adaptations.csv')
films = pd.read_csv('films.csv')
books = pd.read_csv('books.csv')

In [6]:
film_adaptations['target'] = 1

In [7]:
books.columns

Index(['id', 'title', 'authors', 'genres', 'annotation', 'popularity'], dtype='object')

In [8]:
books = books[~books.isna().any(axis=1)].reset_index(drop=True)
films = films[~films.isna().any(axis=1)].reset_index(drop=True)

In [9]:
films_ = films[['id', 'title', 'year']].rename({'id': 'id_film', 'title': 'title_film', 'year': 'year_film'}, axis=1)
books_ = books[['id', 'title', 'authors']].rename({'id': 'id_book', 'title': 'title_book', 'authors': 'authors_book'}, axis=1)

In [10]:
neg_sampler = pd.concat([
    books_.sample(100000, replace=True).reset_index(drop=True),
    films_.sample(100000, replace=True).reset_index(drop=True)
], axis=1)

neg_sampler['target'] = 0

In [14]:
data = pd.concat([film_adaptations, neg_sampler])

In [15]:
data = data.drop_duplicates(subset=['title_book', 'id_book', 'authors_book', 'title_film', 'id_film', 'year_film'])

In [16]:
data = data.sample(frac=1.0).reset_index(drop=True)

#### Добавление фичей

In [22]:
!ls -lh

total 362M
-rw-r--r-- 1 root root  66M Nov 14 18:47 books.csv
-rw-r--r-- 1 root root 148M Nov 14 18:47 description_emb_books.csv
-rw-r--r-- 1 root root  59M Nov 14 18:47 description_emb_films.csv
-rw-r--r-- 1 root root 455K Nov 14 18:47 film_adaptations.csv
-rw-r--r-- 1 root root  28M Nov 14 18:47 films.csv
drwxr-xr-x 1 root root 4.0K Nov 11 14:32 sample_data
-rw-r--r-- 1 root root  63M Nov 14 18:47 tf-idf-sim.csv


In [18]:
description_emb_tf_idf = pd.read_csv('tf-idf-sim.csv')

In [20]:
description_emb_tf_idf[description_emb_tf_idf['book_title'] == 'Как выдать ведьму замуж']

,book_title,book_genre,film_title,film_genre,similarity
0,Как выдать ведьму замуж,"Любовное фэнтези,Магические академии,Юмористич...",Срочно ищу мужа,"русские, мелодрамы",0.244703
1,Как выдать ведьму замуж,"Любовное фэнтези,Магические академии,Юмористич...",Девочка,"драмы, русские, мелодрамы",0.220182
2,Как выдать ведьму замуж,"Любовное фэнтези,Магические академии,Юмористич...",Когда цветет сирень,"русские, мелодрамы",0.219691
3,Как выдать ведьму замуж,"Любовное фэнтези,Магические академии,Юмористич...",Право на любовь,мелодрамы,0.216464
4,Как выдать ведьму замуж,"Любовное фэнтези,Магические академии,Юмористич...",Что скрывает любовь,"русские, криминал, мелодрамы",0.215886


In [25]:
description_emb_books

,id,title,authors,emb_book_0,emb_book_1,emb_book_2,emb_book_3,emb_book_4,emb_book_5,emb_book_6,...,emb_book_302,emb_book_303,emb_book_304,emb_book_305,emb_book_306,emb_book_307,emb_book_308,emb_book_309,emb_book_310,emb_book_311
0,0,Как выдать ведьму замуж,Надежда Соколова,-0.006462,0.025220,-0.002304,-0.07074,-0.056120,-0.011880,0.01041,...,0.015015,0.004210,-0.012230,0.09155,-0.033800,-0.017320,0.005516,-0.058140,-0.00388,-0.04242
1,1,На острие удара,Михаил Михеев,0.070200,0.006077,-0.045870,-0.03818,-0.025570,-0.030530,0.07130,...,-0.036650,0.011320,-0.019120,0.08307,-0.029190,-0.027360,-0.047970,-0.031000,0.05185,-0.04880
2,2,Другой мир. Кровавое дерево. Книга 5,Джордж Гранд,-0.025180,0.046970,-0.070000,-0.10870,-0.024720,-0.042140,0.01224,...,-0.063540,0.081300,0.010190,0.05624,-0.054140,0.031000,-0.049700,-0.034180,0.01130,-0.08167
3,3,Магазин с привидением,Мартин Видмарк,0.020890,0.058400,0.013090,-0.05313,-0.045750,-0.015400,0.02647,...,0.006695,0.045960,-0.015100,0.01874,0.024490,0.012510,-0.081300,0.003529,0.01997,-0.07996
4,4,Один дома 6,Виктория Медведева,0.063350,-0.008280,0.031370,-0.07170,-0.007236,0.031520,-0.04715,...,-0.077940,0.030170,0.020230,0.03745,-0.075600,-0.024570,0.039600,-0.055200,0.00548,-0.03860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57583,59017,Необыкновенная история,Александр Алфёров,0.148400,0.041000,-0.023990,-0.01749,0.026580,-0.038540,0.02502,...,-0.017150,0.014640,-0.007572,-0.04074,-0.077640,-0.027880,-0.037700,0.021700,-0.02406,-0.00776
57584,59018,Ллойс,Дмитрий Панасенко,0.064400,0.000071,-0.005146,-0.09980,-0.046080,0.002897,0.02930,...,0.005676,-0.004780,-0.046940,0.02090,0.006023,-0.058800,-0.062100,-0.051900,0.03784,-0.07310
57585,59019,Иуда Искариот,Леонид Андреев,-0.002730,0.000093,-0.040000,-0.07086,-0.022550,0.013306,0.06067,...,-0.004364,0.052280,0.002650,0.02516,-0.037200,0.016590,-0.043820,-0.035100,-0.04636,-0.01265
57586,59020,Любишь ли ты меня?,Валентина Рыжкова,0.058700,0.025860,0.049040,-0.04556,-0.077300,-0.034270,0.04825,...,-0.015210,-0.054300,-0.023040,0.05750,-0.066650,-0.002111,0.004345,-0.012590,-0.02216,-0.03810


In [24]:
description_emb_books = pd.read_csv('description_emb_books.csv')
description_emb_films = pd.read_csv('description_emb_films.csv')

In [ ]:
description_emb_books = description_emb_books.rename({'id': 'id_book', 'title': 'title_book', 'authors': 'authors_book'}, axis=1)
description_emb_films = description_emb_films.rename({'id': 'id_film', 'title': 'title_film', 'year': 'year_film'}, axis=1)

In [ ]:
print('count data before adding some features', data.shape)
data = data.merge(description_emb_books, on=['id_book', 'title_book', 'authors_book'])
data = data.merge(description_emb_films, on=['id_film', 'title_film', 'year_film'])
print('count data after adding some features', data.shape)

count data before adding some features (100712, 631)
count data after adding some features (100712, 1255)


# Обучение модели

## бейзлайн

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.1, random_state=0, stratify=data['target'])

In [ ]:
cols_exclude_train = [
    'title_book', 'id_book', 'authors_book',
    'title_film', 'id_film', 'year_film'
]

In [ ]:
model = CatBoostClassifier(iterations=200, verbose=0)
model.fit(X_train.drop(cols_exclude_train, axis=1), y_train)

In [ ]:
print("_________TRAIN_METRICS__________")
print(classification_report(y_train, model.predict(X_train.drop(cols_exclude_train, axis=1))))
print()
print("_________TEST_METRICS__________")
print(classification_report(y_test, model.predict(X_test.drop(cols_exclude_train, axis=1))))

_________TRAIN_METRICS__________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     79999
           1       1.00      0.85      0.92       570

    accuracy                           1.00     80569
   macro avg       1.00      0.93      0.96     80569
weighted avg       1.00      1.00      1.00     80569


_________TEST_METRICS__________
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     20001
           1       0.97      0.27      0.43       142

    accuracy                           0.99     20143
   macro avg       0.98      0.64      0.71     20143
weighted avg       0.99      0.99      0.99     20143



In [ ]:
print("_________TRAIN_METRICS__________")
print(classification_report(
    y_train, 
    (model.predict_proba(X_train.drop(cols_exclude_train, axis=1)) > 0.1)[:, 1].astype(int)
))
print()
print("_________TEST_METRICS__________")
print(classification_report(
    y_test, 
    (model.predict_proba(X_test.drop(cols_exclude_train, axis=1)) > 0.1)[:, 1].astype(int)
))

_________TRAIN_METRICS__________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     79999
           1       0.89      0.99      0.94       570

    accuracy                           1.00     80569
   macro avg       0.95      0.99      0.97     80569
weighted avg       1.00      1.00      1.00     80569


_________TEST_METRICS__________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20001
           1       0.63      0.51      0.56       142

    accuracy                           0.99     20143
   macro avg       0.81      0.75      0.78     20143
weighted avg       0.99      0.99      0.99     20143



## Optuna

In [ ]:
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X_train.drop(cols_exclude_train, axis=1), y_train, test_size=0.2)

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    gbm = CatBoostClassifier(**param, verbose=1, iterations=100)

    gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = f1_score(valid_y, pred_labels)
    return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout=60)

[I 2022-11-13 17:48:26,881] A new study created in memory with name: no-name-41f99e18-e494-4fa0-bad6-3fa91c503a7e
[I 2022-11-13 17:48:42,071] Trial 0 finished with value: 0.0 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.0709883886805019, 'depth': 1, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1.954585948267069}. Best is trial 0 with value: 0.0.
[I 2022-11-13 17:49:08,518] Trial 1 finished with value: 0.1733333333333333 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.03692769860358622, 'depth': 7, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 3.136894436085412}. Best is trial 1 with value: 0.1733333333333333.
[I 2022-11-13 17:56:05,261] Trial 2 finished with value: 0.0 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.09495326354987157, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 1 with value: 0.1733333333333333.


In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 3
Best trial:
  Value: 0.1733333333333333
  Params: 
    objective: Logloss
    colsample_bylevel: 0.03692769860358622
    depth: 7
    boosting_type: Plain
    bootstrap_type: Bayesian
    bagging_temperature: 3.136894436085412


In [ ]:
study.

{'objective': 'Logloss',
 'colsample_bylevel': 0.03692769860358622,
 'depth': 7,
 'boosting_type': 'Plain',
 'bootstrap_type': 'Bayesian',
 'bagging_temperature': 3.136894436085412}

# Просмотр результатов

In [ ]:
books_genres = books[['id', 'title', 'genres']].rename({'id': 'id_book', 'title': 'title_book', 'genres': 'genres_book'}, axis=1)
films_genres = films[['id', 'title', 'genres']].rename({'id': 'id_film', 'title': 'title_film', 'genres': 'genres_film'}, axis=1)

X_test_ = (
    X_test
    .copy()
    [['title_book', 'id_book', 'authors_book', 'title_film', 'id_film', 'year_film']]
    .merge(books_genres, on=['id_book', 'title_book'])
    .merge(films_genres, on=['id_film', 'title_film'])
)
X_test_['pred'] = (model.predict_proba(X_test.drop(cols_exclude_train, axis=1)) > 0.1)[:, 1].astype(int)

X_test_[(X_test_.pred == 1) & (y_test == 0)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


,title_book,id_book,authors_book,title_film,id_film,year_film,genres_book,genres_film,pred
745,Летчик для особых поручений,2844,Владислав Крапивин,Все пути ведут в Доннибрук,19922,2018.0,"Детская проза,Сказки","драмы, боевики",1
1335,"Каникулы в Раваншире, или Свадьбы не будет!",15299,Анна Гаврилова,Сломанные судьбы,1462,2013.0,Любовное фэнтези,"русские, мелодрамы",1
1880,Тамплиер. Предательство Святого престола,19543,Юрий Корчевский,Шафер напрокат,20062,2015.0,"Боевая фантастика,Героическая фантастика,Истор...","зарубежные, комедии",1
2779,Шанс на любовь,47089,Аида Янг,Голодные сердца (жестовым языком),854,2018.0,"Короткие любовные романы,Современные любовные ...","драмы, зарубежные, триллеры, мелодрамы",1
3257,Тяжелый свет Куртейна. Синий,53489,Макс Фрай,Зимний замок,16757,2019.0,Городское фэнтези,мелодрамы,1
3433,Похлебка с укропом,8677,Владислав Крапивин,Тихая гавань,11650,2013.0,Детская проза,"драмы, триллеры, мелодрамы",1
4281,Стихотворения о родной природе,23030,Сборник,Сам я - вятский уроженец,18293,1992.0,"Русская поэзия,Стихи и поэзия","русские, фэнтези, комедии",1
4426,Влиятельные джентльмены,37570,"Антон Казаков,Дмитрий Казанцев",Девушка с косой,19345,2017.0,"Детективная фантастика,Историческая фантастика...","семейное, русские, комедии",1
4658,Дерзкое предложение дебютантки,38109,Энни Берроуз,Везучий случай,10262,2017.0,"Зарубежные любовные романы,Исторические любовн...",комедии,1
5214,Преступления любви. Половая психопатия,5058,Рихард Крафт-Эбинг,Брат 2,7464,2000.0,"Зарубежная образовательная литература,Зарубежн...","боевики, русские, криминал",1
